# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'08-16-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'08-16-2020'

In [4]:
# Check out the first 5 rows to make sure the data loaded correctly
jh_live_global.head(20)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-08-17 04:27:20,33.939110,67.709953,37596,1375,27166,9055.0,Afghanistan,96.577452,3.657304
1,NaN,NaN,NaN,Albania,2020-08-17 04:27:20,41.153300,20.168300,7380,228,3794,3358.0,Albania,256.445896,3.089431
2,NaN,NaN,NaN,Algeria,2020-08-17 04:27:20,28.033900,1.659600,38583,1370,27017,10196.0,Algeria,87.986505,3.550787
3,NaN,NaN,NaN,Andorra,2020-08-17 04:27:20,42.506300,1.521800,989,53,863,73.0,Andorra,1280.010354,5.358948
4,NaN,NaN,NaN,Angola,2020-08-17 04:27:20,-11.202700,17.873900,1906,88,628,1190.0,Angola,5.799259,4.616999
5,NaN,NaN,NaN,Antigua and Barbuda,2020-08-17 04:27:20,17.060800,-61.796400,93,3,88,2.0,Antigua and Barbuda,94.967731,3.225806
6,NaN,NaN,NaN,Argentina,2020-08-17 04:27:20,-38.416100,-63.616700,294569,5703,211702,77164.0,Argentina,651.762221,1.936049
7,NaN,NaN,NaN,Armenia,2020-08-17 04:27:20,40.069100,45.038200,41663,818,34584,6261.0,Armenia,1405.997636,1.963373
8,NaN,NaN,Australian Capital Territory,Australia,2020-08-17 04:27:20,-35.473500,149.012400,113,3,110,0.0,"Australian Capital Territory, Australia",26.395702,2.654867
9,NaN,NaN,New South Wales,Australia,2020-08-17 04:27:20,-33.868800,151.209300,3957,52,2994,911.0,"New South Wales, Australia",48.743533,1.314127


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,37015,37054,37054,37162,37269,37345,37424,37431,37551,37596
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,6151,6275,6411,6536,6676,6817,6971,7117,7260,7380
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,34155,34693,35160,35712,36204,36699,37187,37664,38133,38583


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1307,1312,1312,1328,1344,1354,1363,1363,1370,1375
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,189,193,199,200,205,208,213,219,225,228
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1282,1293,1302,1312,1322,1333,1341,1351,1360,1370


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,25903,25960,25960,26228,26415,26694,26714,26714,27166,27166
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3227,3268,3342,3379,3480,3552,3616,3695,3746,3794
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,23667,24083,24506,24920,25263,25627,26004,26308,26644,27017


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1065,1086,1169,1174,1162,1188,1196,1196,1206,1218
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,3443,3502,3544,3651,3670,3710,3744,3791,3839,3877
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,569,573,575,576,579,581,586,590,598,604


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,21,21,21,21,22,22,22,22,22,22
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,23,24,25,25,29,29,29,29,29,29
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,5,5,5,5,6,6,6,6,6,6


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
631,1001.0,Autauga,Alabama,US,2020-08-17 04:27:20,32.539527,-86.644082,1218,22,0,1196.0,"Autauga, Alabama, US",2180.099876,1.806240
668,1075.0,Lamar,Alabama,US,2020-08-17 04:27:20,33.779950,-88.096680,240,2,0,238.0,"Lamar, Alabama, US",1738.500543,0.833333
669,1077.0,Lauderdale,Alabama,US,2020-08-17 04:27:20,34.901719,-87.656247,1238,20,0,1218.0,"Lauderdale, Alabama, US",1335.073170,1.615509


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,5403213,170052,1833067
Brazil,3340197,107852,2655017
India,2647663,50921,1919842


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,5403213,170052,1833067,3400094,2020-08-17 04:27:20,37.913308,-91.347527
Brazil,3340197,107852,2655017,577328,2020-08-17 04:27:20,-12.669522,-48.480493
India,2647663,50921,1919842,676900,2020-08-17 04:27:20,23.088275,81.806127
Russia,920719,15653,731444,173622,2020-08-17 04:27:20,54.546312,62.120860
South Africa,587345,11839,472377,103129,2020-08-17 04:27:20,-30.559500,22.937500


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,623873,11243,0
Florida,573416,9452,0
Texas,555394,10396,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,623873,11243,0,612630,2020-08-17 04:27:20,37.843962,-120.728594
Florida,573416,9452,0,563964,2020-08-17 04:27:20,28.940755,-82.700744
Texas,555394,10396,0,544998,2020-08-17 04:27:20,31.660643,-98.653069
New York,425508,32840,0,392668,2020-08-17 04:27:20,42.671593,-75.579694
Georgia,237030,4702,0,232328,2020-08-17 04:27:20,32.808532,-83.577430


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
New York,New York City,230223,23628,0
California,Los Angeles,221971,5254,0
Florida,Miami-Dade,145307,2057,0
Arizona,Maricopa,129385,2596,0
Illinois,Cook,115960,4962,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
New York,New York City,230223,23628,0,206595,2020-08-17 04:27:20,40.767273,-73.971526,36061.0
California,Los Angeles,221971,5254,0,216717,2020-08-17 04:27:20,34.308284,-118.228241,6037.0
Florida,Miami-Dade,145307,2057,0,143250,2020-08-17 04:27:20,25.611236,-80.551706,12086.0
Arizona,Maricopa,129385,2596,0,126789,2020-08-17 04:27:20,33.348359,-112.491815,4013.0
Illinois,Cook,115960,4962,0,110998,2020-08-17 04:27:20,41.841448,-87.816588,17031.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-14,37431,7117,37664,989,1852,93,282437,41299,23035,22876,...,318190,1421,33821,31381,930,15834,10,1858,9021,5072
2020-08-15,37551,7260,38133,989,1879,93,289100,41495,23287,23179,...,319232,1434,34528,32607,951,16153,10,1858,9186,5176
2020-08-16,37596,7380,38583,989,1906,93,294569,41663,23558,23370,...,320343,1440,35329,33755,964,16534,10,1869,9343,5261


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-14,1363,219,1351,53,86,3,5527,814,379,725,...,46791,38,220,266,22,106,1,528,256,128
2020-08-15,1370,225,1360,53,86,3,5637,817,396,728,...,46791,38,225,276,24,108,1,528,260,130
2020-08-16,1375,228,1370,53,88,3,5703,818,421,728,...,46791,38,232,281,24,110,1,530,260,132


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-14,26714,3695,26308,863,584,83,199005,34164,13632,20499,...,1481,1182,27825,21580,437,9382,8,1009,7586,1998
2020-08-15,27166,3746,26644,863,628,83,205697,34484,13633,20627,...,1486,1194,29328,21747,447,9388,8,1013,8065,2047
2020-08-16,27166,3794,27017,863,628,88,211702,34584,14083,20681,...,1486,1200,30973,22700,456,9838,8,1013,8412,2092


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1065,1086,1169,1174,1162,1188,1196,1196,1206,1218
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,217,220,222,229,230,230,237,237,239,240
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,1139,1157,1167,1189,1190,1203,1213,1219,1231,1238
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,344,348,349,351,354,356,360,364,367,370
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,2628,2660,2678,2700,2708,2724,2736,2735,2760,2776


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-14,106309,4073,0,191721,52392,613101,52519,50897,16340,49,...,130458,539673,45976,1501,704,104838,66139,8277,64227,3183
2020-08-15,107580,4156,0,192654,51992,617994,52819,50897,16396,49,...,131747,546552,46321,1509,734,105750,66885,8462,65056,3227
2020-08-16,108433,4259,0,193537,52665,623873,53157,50897,16451,49,...,133708,555394,46652,1515,741,106687,67461,8567,65741,3286


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-08-14        1196    3791     590  465    848     494    779    1867   
2020-08-15        1206    3839     598  469    854     495    788    1903   
2020-08-16        1218    3877     604  469    860     496    792    1919   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-14          852      293  ...     144      6       78       40   
2020-08-15          854      299  ...     144      6       81       40   
2020-08-16          854      302  ...     144      6       83       40   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-14            271   383   277          0       96      5  
2020-08-15            275   385   276          0       97      7  
2020-08-16            276   389   276          0       97      7  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-14,1893,27,0,4423,587,11148,1888,4453,593,0,...,1326,10078,360,58,9,2370,1755,157,1025,30
2020-08-15,1896,28,0,4492,600,11218,1896,4453,593,0,...,1345,10268,363,58,9,2381,1766,160,1038,30
2020-08-16,1898,28,0,4506,599,11243,1896,4453,593,0,...,1366,10396,363,58,9,2381,1781,160,1039,30


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-08-14          22      29       6    5      5      11     36      19   
2020-08-15          22      29       6    5      5      11     36      19   
2020-08-16          22      29       6    5      5      11     36      19   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-14           38        8  ...       0      0        0        0   
2020-08-15           38        8  ...       0      0        0        0   
2020-08-16           38        8  ...       0      0        0        0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-14              0     0     0         29        0      0  
2020-08-15              0     0     0         29        0      0  
2020-08-16              0     0     0         29        0      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-14,0.000187,0.020944,0.012827,0.008155,0.020386,0.01087,0.023056,0.006728,0.012884,0.012481,...,0.004613,0.008517,0.014945,0.033323,0.020856,0.022142,0.0,0.005956,0.041325,0.016433
2020-08-15,0.003206,0.020093,0.012452,0.000000,0.014579,0.00000,0.023591,0.004746,0.010940,0.013245,...,0.003275,0.009148,0.020904,0.039068,0.022581,0.020147,0.0,0.000000,0.018291,0.020505
2020-08-16,0.001198,0.016529,0.011801,0.000000,0.014369,0.00000,0.018917,0.004049,0.011637,0.008240,...,0.003480,0.004184,0.023199,0.035207,0.013670,0.023587,0.0,0.005920,0.017091,0.016422


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-14,0.000000,0.028169,0.007457,0.0,0.075000,0.0,0.030772,0.006180,0.010667,0.000000,...,0.0,0.027027,0.018519,0.027027,0.047619,0.000000,0.0,0.000000,0.040650,0.000000
2020-08-15,0.005136,0.027397,0.006662,0.0,0.000000,0.0,0.019902,0.003686,0.044855,0.004138,...,0.0,0.000000,0.022727,0.037594,0.090909,0.018868,0.0,0.000000,0.015625,0.015625
2020-08-16,0.003650,0.013333,0.007353,0.0,0.023256,0.0,0.011708,0.001224,0.063131,0.000000,...,0.0,0.000000,0.031111,0.018116,0.000000,0.018519,0.0,0.003788,0.000000,0.015385


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-14,0.00000,0.021847,0.011691,0.005828,0.012132,0.092105,0.034147,0.007877,0.048535,0.007520,...,0.001352,0.001695,0.022489,0.009119,0.028235,0.021337,0.0,0.063224,0.024997,0.036845
2020-08-15,0.01692,0.013802,0.012772,0.000000,0.075342,0.000000,0.033627,0.009367,0.000073,0.006244,...,0.003376,0.010152,0.054016,0.007739,0.022883,0.000640,0.0,0.003964,0.063143,0.024525
2020-08-16,0.00000,0.012814,0.013999,0.000000,0.000000,0.060241,0.029193,0.002900,0.033008,0.002618,...,0.000000,0.005025,0.056090,0.043822,0.020134,0.047934,0.0,0.000000,0.043025,0.021983


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-14,0.007124,0.027757,NaN,0.004859,0.012093,0.025280,0.006111,0.002265,0.023361,0.0,...,0.015150,0.016607,0.012152,0.011456,0.032258,0.011735,0.012244,0.013965,0.016154,0.020519
2020-08-15,0.011956,0.020378,NaN,0.004866,-0.007635,0.007981,0.005712,0.000000,0.003427,0.0,...,0.009881,0.012747,0.007504,0.005330,0.042614,0.008699,0.011279,0.022351,0.012907,0.013823
2020-08-16,0.007929,0.024783,NaN,0.004583,0.012944,0.009513,0.006399,0.000000,0.003354,0.0,...,0.014885,0.016178,0.007146,0.003976,0.009537,0.008861,0.008612,0.012408,0.010529,0.018283


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-08-14      0.000000  0.012553  0.006826  0.017505  0.015569  0.002028   
2020-08-15      0.008361  0.012662  0.013559  0.008602  0.007075  0.002024   
2020-08-16      0.009950  0.009898  0.010033  0.000000  0.007026  0.002020   

Province_State                                          ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park Platte   
2020-08-14      0.005161  0.007555  0.004717  0.017361  ...  0.035971    0.0   
2020-08-15      0.011553  0.019282  0.002347  0.020478  ...  0.000000    0.0   
2020-08-16      0.005076  0.008408  0.000000  0.010033  ...  0.000000    0.0   

Province_State                                                               \
Admin2          Sheridan Sublette Sweetwater     Teton     Uinta Unassigned   
2020-08-14      0.054054      0.0   0.014981  0.021333 -0.003597        NaN   
2020-08-15      0.038462      0.0   0.014760  0.005222 -0.003610        NaN   
2020-08-16      0.024691      0.0   0.003636  0.010390  0.000000        NaN   

Province_State                   
Admin2          Washakie Weston  
2020-08-14      0.116279    0.0  
2020-08-15      0.010417    0.4  
2020-08-16      0.000000    0.0  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-14,0.001587,0.000000,NaN,0.009126,0.008591,0.025575,0.003188,0.000674,0.0,NaN,...,0.009901,0.031420,0.019830,0.0,0.0,0.002962,0.010945,0.026144,0.006876,0.0
2020-08-15,0.001585,0.037037,NaN,0.015600,0.022147,0.006279,0.004237,0.000000,0.0,NaN,...,0.014329,0.018853,0.008333,0.0,0.0,0.004641,0.006268,0.019108,0.012683,0.0
2020-08-16,0.001055,0.000000,NaN,0.003117,-0.001667,0.002229,0.000000,0.000000,0.0,NaN,...,0.015613,0.012466,0.000000,0.0,0.0,0.000000,0.008494,0.000000,0.000963,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-08-14         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2020-08-15         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2020-08-16         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-14          0.0      0.0  ...     NaN    NaN      NaN      NaN   
2020-08-15          0.0      0.0  ...     NaN    NaN      NaN      NaN   
2020-08-16          0.0      0.0  ...     NaN    NaN      NaN      NaN   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-14            NaN   NaN   NaN        0.0      NaN    NaN  
2020-08-15            NaN   NaN   NaN        0.0      NaN    NaN  
2020-08-16            NaN   NaN   NaN        0.0      NaN    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-14,0.001175,0.021378,0.013260,0.00724,0.023393,0.005438,0.025609,0.005817,0.014244,0.009710,...,0.004038,0.009385,0.018622,0.037768,0.023621,0.021575,6.167906e-18,0.005068,0.030895,0.017379
2020-08-15,0.002190,0.020735,0.012856,0.00362,0.018986,0.002719,0.024600,0.005282,0.012592,0.011478,...,0.003656,0.009267,0.019763,0.038418,0.023101,0.020861,3.083953e-18,0.002534,0.024593,0.018942
2020-08-16,0.001694,0.018632,0.012328,0.00181,0.016678,0.001360,0.021759,0.004665,0.012115,0.009859,...,0.003568,0.006726,0.021481,0.036813,0.018385,0.022224,1.541976e-18,0.004227,0.020842,0.017682


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-14,0.003795,0.024439,0.007225,0.002404,0.042143,1.895485e-37,0.032742,0.005476,0.025766,0.000622,...,0.000338,0.013531,0.022791,0.034790,0.095340,0.007578,0.0,0.002116,0.026606,0.035203
2020-08-15,0.004465,0.025918,0.006943,0.001202,0.021071,9.477423e-38,0.026322,0.004581,0.035310,0.002380,...,0.000169,0.006765,0.022759,0.036192,0.093125,0.013223,0.0,0.001058,0.021116,0.025414
2020-08-16,0.004057,0.019626,0.007148,0.000601,0.022164,4.738712e-38,0.019015,0.002902,0.049221,0.001190,...,0.000084,0.003383,0.026935,0.027154,0.046562,0.015871,0.0,0.002423,0.010558,0.020399


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-14,0.002314,0.020825,0.013273,0.00624,0.007929,0.046095,0.070873,0.009637,0.034910,0.006296,...,0.002884,0.006754,0.041427,0.036372,0.020819,0.040806,3.614007e-20,0.038049,0.026136,0.076602
2020-08-15,0.009617,0.017314,0.013023,0.00312,0.041636,0.023048,0.052250,0.009502,0.017492,0.006270,...,0.003130,0.008453,0.047721,0.022055,0.021851,0.020723,1.807004e-20,0.021007,0.044640,0.050563
2020-08-16,0.004809,0.015064,0.013511,0.00156,0.020818,0.041644,0.040722,0.006201,0.025250,0.004444,...,0.001565,0.006739,0.051906,0.032939,0.020993,0.034328,9.035018e-21,0.010503,0.043832,0.036273


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-14,0.008014,0.023244,NaN,0.005359,0.012370,0.019799,0.007030,0.001870,0.015935,1.192594e-46,...,0.014598,0.014894,0.009858,0.007856,0.051476,0.010695,0.010786,0.015487,0.014273,0.014346
2020-08-15,0.009985,0.021811,NaN,0.005113,0.002368,0.013890,0.006371,0.000935,0.009681,5.962972e-47,...,0.012239,0.013820,0.008681,0.006593,0.047045,0.009697,0.011032,0.018919,0.013590,0.014085
2020-08-16,0.008957,0.023297,NaN,0.004848,0.007656,0.011702,0.006385,0.000468,0.006518,2.981486e-47,...,0.013562,0.014999,0.007913,0.005285,0.028291,0.009279,0.009822,0.015664,0.012060,0.016184


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-08-14      0.005428  0.011661  0.006522  0.014893  0.013484  0.006789   
2020-08-15      0.006895  0.012161  0.010041  0.011748  0.010280  0.004407   
2020-08-16      0.008422  0.011030  0.010037  0.005874  0.008653  0.003214   

Province_State                                          ...   Wyoming          \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park  Platte   
2020-08-14      0.004221  0.009277  0.002925  0.019123  ...  0.026109  0.0500   
2020-08-15      0.007887  0.014280  0.002636  0.019801  ...  0.013055  0.0250   
2020-08-16      0.006482  0.011344  0.001318  0.014917  ...  0.006527  0.0125   

Province_State                                                                \
Admin2          Sheridan  Sublette Sweetwater     Teton     Uinta Unassigned   
2020-08-14      0.031033  0.003232   0.009970  0.012576 -0.001608       -1.0   
2020-08-15      0.034748  0.001616   0.012365  0.008899 -0.002609       -1.0   
2020-08-16      0.029719  0.000808   0.008001  0.009644 -0.001304       -1.0   

Province_State                      
Admin2          Washakie    Weston  
2020-08-14      0.079201 -0.004166  
2020-08-15      0.044809  0.197917  
2020-08-16      0.022405  0.098958  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-14,0.006784,0.005134,NaN,0.012049,0.012468,0.019174,0.003087,0.000531,0.000695,NaN,...,0.013897,0.027053,0.013775,0.000035,0.000245,0.003664,0.008827,0.022610,0.006451,0.011007
2020-08-15,0.004184,0.021086,NaN,0.013825,0.017307,0.012726,0.003662,0.000265,0.000348,NaN,...,0.014113,0.022953,0.011054,0.000017,0.000123,0.004153,0.007548,0.020859,0.009567,0.005504
2020-08-16,0.002620,0.010543,NaN,0.008471,0.007820,0.007477,0.001831,0.000133,0.000174,NaN,...,0.014863,0.017710,0.005527,0.000009,0.000061,0.002076,0.008021,0.010430,0.005265,0.002752


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                          \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   
2020-08-14      0.003025  0.011011  0.012504  0.002340  0.018244   
2020-08-15      0.001513  0.005505  0.006252  0.001170  0.009122   
2020-08-16      0.000756  0.002753  0.003126  0.000585  0.004561   

Province_State                                                            ...  \
Admin2               Bullock    Butler   Calhoun      Chambers  Cherokee  ...   
2020-08-14      5.960489e-09  0.000223  0.018772  7.453150e-08  0.004464  ...   
2020-08-15      2.980245e-09  0.000112  0.009386  3.726575e-08  0.002232  ...   
2020-08-16      1.490122e-09  0.000056  0.004693  1.863288e-08  0.001116  ...   

Province_State Wyoming                                                  \
Admin2            Park Platte Sheridan Sublette Sweetwater Teton Uinta   
2020-08-14         NaN    NaN      NaN      NaN        NaN   NaN   NaN   
2020-08-15         NaN    NaN      NaN      NaN        NaN   NaN   NaN   
2020-08-16         NaN    NaN      NaN      NaN        NaN   NaN   NaN   

Province_State                             
Admin2         Unassigned Washakie Weston  
2020-08-14       0.011404      NaN    NaN  
2020-08-15       0.005702      NaN    NaN  
2020-08-16       0.002851      NaN    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200816,AK,5060,296455,NaN,42.0,NaN,NaN,NaN,3.0,...,301515,0,0,f63b8b46c05abed89db65c7e45b4494f53925fe8,0,0,0,0,0,NaN
1,20200816,AL,108433,737991,NaN,1291.0,12607.0,NaN,1316.0,NaN,...,846424,2,0,bf0c020f6423a114c61f4e186d87590f16d2d70e,0,0,0,0,0,NaN
2,20200816,AR,52665,569385,NaN,478.0,3570.0,NaN,NaN,120.0,...,622050,-1,8,6a4419b77c720a48cae84c9bef21997f9908a547,0,0,0,0,0,NaN
3,20200816,AS,0,1514,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,47b4a5336ce028024c6c6c7f708cd84f5c78729a,0,0,0,0,0,NaN
4,20200816,AZ,193537,898437,NaN,1208.0,20755.0,417.0,NaN,267.0,...,1091974,14,-40,09fc1e80117358862ddcab47132c625db6c67856,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200816,AK,5060.0,296455.0,NaN,42.0,NaN,NaN,NaN,3.0,...,301515,0,0,f63b8b46c05abed89db65c7e45b4494f53925fe8,0,0,0,0,0,NaN
1,20200816,AL,108433.0,737991.0,NaN,1291.0,12607.0,NaN,1316.0,NaN,...,846424,2,0,bf0c020f6423a114c61f4e186d87590f16d2d70e,0,0,0,0,0,NaN
2,20200816,AR,52665.0,569385.0,NaN,478.0,3570.0,NaN,NaN,120.0,...,622050,-1,8,6a4419b77c720a48cae84c9bef21997f9908a547,0,0,0,0,0,NaN
3,20200816,AS,0.0,1514.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,47b4a5336ce028024c6c6c7f708cd84f5c78729a,0,0,0,0,0,NaN
4,20200816,AZ,193537.0,898437.0,NaN,1208.0,20755.0,417.0,NaN,267.0,...,1091974,14,-40,09fc1e80117358862ddcab47132c625db6c67856,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-08-16,AK,5060,296455,NaN,42.0,NaN,NaN,NaN,3.0,NaN,...,301515,0,0,f63b8b46c05abed89db65c7e45b4494f53925fe8,0,0,0,0,0,NaN
2020-08-16,AL,108433,737991,NaN,1291.0,12607.0,NaN,1316.0,NaN,720.0,...,846424,2,0,bf0c020f6423a114c61f4e186d87590f16d2d70e,0,0,0,0,0,NaN
2020-08-16,AR,52665,569385,NaN,478.0,3570.0,NaN,NaN,120.0,460.0,...,622050,-1,8,6a4419b77c720a48cae84c9bef21997f9908a547,0,0,0,0,0,NaN
2020-08-16,AS,0,1514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,47b4a5336ce028024c6c6c7f708cd84f5c78729a,0,0,0,0,0,NaN
2020-08-16,AZ,193537,898437,NaN,1208.0,20755.0,417.0,NaN,267.0,NaN,...,1091974,14,-40,09fc1e80117358862ddcab47132c625db6c67856,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-08-16,AK,5060.0,296455.0,NaN,42.0,NaN,NaN,NaN,3.0,NaN,...,301515,0,0,f63b8b46c05abed89db65c7e45b4494f53925fe8,0,0,0,0,0,NaN
2020-08-16,AL,108433.0,737991.0,NaN,1291.0,12607.0,NaN,1316.0,NaN,720.0,...,846424,2,0,bf0c020f6423a114c61f4e186d87590f16d2d70e,0,0,0,0,0,NaN
2020-08-16,AR,52665.0,569385.0,NaN,478.0,3570.0,NaN,NaN,120.0,460.0,...,622050,-1,8,6a4419b77c720a48cae84c9bef21997f9908a547,0,0,0,0,0,NaN
2020-08-16,AS,0.0,1514.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,47b4a5336ce028024c6c6c7f708cd84f5c78729a,0,0,0,0,0,NaN
2020-08-16,AZ,193537.0,898437.0,NaN,1208.0,20755.0,417.0,NaN,267.0,NaN,...,1091974,14,-40,09fc1e80117358862ddcab47132c625db6c67856,0,0,0,0,0,NaN


# TODO - CONTINUE FROM HERE